# Implementation of a CNN Fire module for SqueezeNet
*by Marvin Bertin*
<img src="../../images/keras-tensorflow-logo.jpg" width="400">

# Imports

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np

2023-07-24 15:41:57.966969: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-24 15:41:58.010982: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-24 15:41:58.012132: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-24 15:41:59.246755: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
layers =  tf.keras.layers
models = tf.keras.models

# Implementation of Fire Module

<img src="../../images/fire_module.png" width="600">

In [4]:
def fire_module(x, fire_id, squeeze=16, expand=64):
    sq1x1 = "squeeze1x1"
    exp1x1 = "expand1x1"
    exp3x3 = "expand3x3"
    relu = "relu_"
    s_id = 'fire' + str(fire_id) + '/'

    # Squeeze layer
    x = layers.Convolution2D(squeeze, (1,1), padding='valid', name=s_id + sq1x1)(x)
    x = layers.Activation('relu', name=s_id + relu + sq1x1)

    # Expand layer 1x1 filters
    left = layers.Convolution2D(expand, (1,1), padding='valid', name=s_id + exp1x1)(x)
    left = layers.Activation('relu', name=s_id + relu + exp1x1)(left)

    # Expand layer 3x3 filters
    right = layers.Convolution2D(expand, (3,3), padding='same', name=s_id + exp3x3)(x)
    right = layers.Activation('relu', name=s_id + relu + exp3x3)(right)

    # concatenate outputs
    x = layers.concatenate([left, right], axis=3, name=s_id + 'concat')

    
    return x

# Implementation of SqueezeNet

In [5]:
def SqueezeNet(input_shape=(32,32,3), classes=10):
    
    img_input = layers.Input(shape=input_shape)
    
    x = layers.Convolution2D(64, (3, 3), strides=(2, 2), padding='valid', name='conv1')(img_input)
    x = layers.Activation('relu', name='relu_conv1')(x)
#     x = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool1')(x)

    x = fire_module(x, fire_id=2, squeeze=16, expand=64)
    x = fire_module(x, fire_id=3, squeeze=16, expand=64)
    x = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool3')(x)

    x = fire_module(x, fire_id=4, squeeze=32, expand=128)
    x = fire_module(x, fire_id=5, squeeze=32, expand=128)
    x = layers.Dropout(0.5, name='drop9')(x)

    x = layers.Convolution2D(classes, (1, 1), padding='valid', name='conv10')(x)
    x = layers.Activation('relu', name='relu_conv10')(x)
    x = layers.GlobalAveragePooling2D()(x)
    out = layers.Activation('softmax', name='loss')(x)

    model = models.Model(img_input, out, name='squeezenet')

    return model

# Inspect SqueezeNet Architecture

In [6]:
squeeze_net = SqueezeNet()
squeeze_net.summary()

TypeError: Inputs to a layer should be tensors. Got '<keras.layers.core.activation.Activation object at 0x55556476df70>' (of type <class 'keras.layers.core.activation.Activation'>) as input for layer 'fire2/expand1x1'.

## Next Lesson
### Train and Evaluate SqueezeNet
- Image classification task with Cifar10

<img src="../../images/divider.png" width="100">